Starting point


Cleaned | D3QN

In [16]:
from typing import List, Tuple, Optional
import numpy as np

class InventoryEnv:
    """
    Inventory management environment for 3 products with volume constraints, lead times, and stochastic or deterministic demand.

    This environment simulates:
    - Warehouse inventory evolution with lead-time-based ordering.
    - Daily customer demand and fulfillment.
    - Cost computation due to holding, ordering, and stockouts.

    Attributes:
        volume_capacity (float): Max warehouse volume capacity.
        initial_inventory (List[int]): Initial stock for each product.
        product_volumes (List[float]): Volume per unit of each product.
        holding_cost_per_volume (float): Cost per unit volume per day for storing inventory.
        stockout_costs (List[float]): Penalty per unit of unfulfilled demand for each product.
        ordering_costs (List[float]): Fixed cost per order placed for each product.
        discard_costs (List[float]): Cost per unit discarded due to over-capacity.
        lead_times (List[int]): Days before an order arrives for each product.
        simulation_days (int): Episode length in days.
        demand_sequences (Optional[List[List[int]]]): Predefined demand for evaluation.
        demand_lambda (List[float]): Poisson mean for training demand generation.
    """

    def __init__(
        self,
        volume_capacity: float = 1000.0,
        initial_inventory: List[int] = [100, 100, 100],
        product_volumes: List[float] = [2.0, 3.0, 1.5],
        holding_cost_per_volume: float = 5.0,  # Updated holding cost
        stockout_costs: List[float] = [400.0, 500.0, 300.0],
        ordering_costs: List[float] = [80.0, 200.0, 120.0],
        discard_costs: List[float] = [200.0, 250.0, 150.0],  # New discard penalties
        lead_times: List[int] = [3, 2, 1],
        simulation_days: int = 50,
        demand_sequences: Optional[List[List[int]]] = None,
        demand_lambda: List[float] = [30, 25, 35],
        seed: int = 42
    ):
        self.volume_capacity = volume_capacity
        self.initial_inventory = initial_inventory[:]
        self.product_volumes = product_volumes
        self.holding_cost_per_volume = holding_cost_per_volume
        self.stockout_costs = stockout_costs
        self.ordering_costs = ordering_costs
        self.discard_costs = discard_costs
        self.lead_times = lead_times
        self.simulation_days = simulation_days
        self.demand_sequences = demand_sequences
        self.demand_lambda = demand_lambda
        self.random_state = np.random.RandomState(seed)

        self.last_orders = [0, 0, 0] # Initialize last orders for stability penalty
        self.reset()

    def reset(self) -> List[int]:
        """
        Reset environment to initial state for a new episode.
        Returns the initial observation state.
        """
        self.day = 0
        self.inventory = self.initial_inventory[:] # Resets current inventory to initial inventory
        self.pending_orders = [[] for _ in range(len(self.initial_inventory))]  # list of orders to be delivered (day_due, quantity)
        self.last_orders = [0, 0, 0] # Reset last orders
        return self._get_state() # Returns initial state of the environment

    def step(self, action: List[int]) -> Tuple[List[int], float, bool, dict]:
        """
        Executes one simulation step.

        Args:
            action (List[int]): List of order quantities for each product. Each value must be in {0, 10, ..., 100}.

        Returns:
            state (List[int]): Updated state after taking the action.
            reward (float): Scaled negative cost for the step.
            done (bool): True if the episode is over.
            info (dict): Additional information (cost breakdown, demand, fulfillment).
        """
        assert all(a in range(0, 101, 10) for a in action), "Actions must be in {0, 10, ..., 100}" # Invalid actions are rejected

        # 1. Receive due orders and add them to current inventory
        for i in range(3):
            arrivals = [qty for due, qty in self.pending_orders[i] if due == self.day]
            self.inventory[i] += sum(arrivals)
            self.pending_orders[i] = [(due, qty) for due, qty in self.pending_orders[i] if due > self.day]

        # 2. Place new orders and add them to pending orders
        order_cost = 0
        for i in range(3):
            if action[i] > 0:
                order_cost += self.ordering_costs[i]
                self.pending_orders[i].append((self.day + self.lead_times[i], action[i]))

        # 3. Generate demand if not provided
        if self.demand_sequences:
            demand = self.demand_sequences[self.day]
        else:
            demand = [self.random_state.poisson(lam) for lam in self.demand_lambda]

        # 4. Enforce volume capacity and compute discards
        total_volume = sum(self.inventory[i] * self.product_volumes[i] for i in range(3))
        discarded = [0, 0, 0]
        if total_volume > self.volume_capacity:
            overflow = total_volume - self.volume_capacity
            # discard from highest-volume items first
            for i in sorted(range(3), key=lambda j: self.product_volumes[j], reverse=True):
                max_remove = int(overflow // self.product_volumes[i])
                remove_qty = min(max_remove, self.inventory[i])
                discarded[i] = remove_qty
                self.inventory[i] -= remove_qty
                overflow -= remove_qty * self.product_volumes[i]
                if overflow <= 0:
                    break

        # 5. Fulfill demand and compute stockouts
        fulfilled = [min(self.inventory[i], demand[i]) for i in range(3)]
        unfulfilled = [demand[i] - fulfilled[i] for i in range(3)]
        self.inventory = [self.inventory[i] - fulfilled[i] for i in range(3)]

        # 6. Compute costs and reward
        holding_cost = sum(self.inventory[i] * self.product_volumes[i] * self.holding_cost_per_volume for i in range(3))
        stockout_cost = sum(unfulfilled[i] * self.stockout_costs[i] for i in range(3))
        discard_cost = sum(discarded[i] * self.discard_costs[i] for i in range(3))
        total_cost = holding_cost + stockout_cost + order_cost + discard_cost
        reward = - total_cost / 100.0  # scaled for stability

        # Reward Shaping (only during training)
        # Note: This part is typically done in the agent's learning step, not the environment step.
        # However, for simplicity and to demonstrate the concept, we'll add it here.
        # In a real-world scenario, you'd pass a flag to step or handle shaping in the agent.

        # Inventory balance bonus: Encourage inventory near a target level (e.g., 50 for each product)
        target_inventory = [50, 50, 50]
        inventory_balance_bonus = 0
        for i in range(3):
             inventory_balance_bonus -= abs(self.inventory[i] - target_inventory[i]) * 0.1 # Small penalty for deviation


        # Order stability penalty: Penalize large swings in daily order quantity
        order_stability_penalty = 0
        for i in range(3):
            order_stability_penalty -= abs(action[i] - self.last_orders[i]) * 0.05 # Small penalty for order changes

        self.last_orders = action # Update last orders


        # Stockout-free reward: Bonus for zero unfulfilled demand
        stockout_free_bonus = 0
        if all(u == 0 for u in unfulfilled):
            stockout_free_bonus += 1.0 # Small bonus for no stockouts

        reward += inventory_balance_bonus + order_stability_penalty + stockout_free_bonus


        # 7. Update state
        self.day += 1
        done = self.day >= self.simulation_days # True if episode ends
        info = {
            "day": self.day,
            "inventory": self.inventory[:],
            "fulfilled": fulfilled,
            "unfulfilled": unfulfilled,
            "order_cost": order_cost,
            "holding_cost": holding_cost,
            "stockout_cost": stockout_cost,
            "discard_cost": discard_cost,
            "total_cost": total_cost
        }

        return self._get_state(), reward, done, info

    def _get_state(self) -> List[int]:
        """
        Constructs the state vector including inventory levels and outstanding orders.

        Returns:
            List[int]: State representation with 7 variables
        """
        outstanding_orders = [sum(qty for _, qty in self.pending_orders[i]) for i in range(3)]
        return self.inventory + outstanding_orders + [self.day]


import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

class DuelingQNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(DuelingQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

        self.value_stream = nn.Linear(hidden_size, 1)
        self.advantage_stream = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        value = self.value_stream(x)
        advantage = self.advantage_stream(x)

        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

class DQNAgent:
    def __init__(self, state_size, action_size, gamma=0.99, lr=5e-4, # Increased learning rate
                 batch_size=128, buffer_size=100000, epsilon_start=1.0, # Increased batch size and buffer size
                 epsilon_end=0.01, epsilon_decay=0.99, alpha=0.6, beta_start=0.4, beta_frames=1000): # Added alpha and beta params
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.batch_size = batch_size
        # Replace deque with PrioritizedReplayBuffer
        self.memory = PrioritizedReplayBuffer(buffer_size, alpha)
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay

        self.beta = beta_start # For importance sampling
        self.beta_increment_per_frame = (1.0 - beta_start) / beta_frames # Beta annealing

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork = DuelingQNetwork(state_size, action_size).to(self.device)
        self.target_network = DuelingQNetwork(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.qnetwork.parameters(), lr=lr) # Using Adam optimizer

        # For state normalization (reset for new agent)
        self.state_mean = np.zeros(state_size)
        self.state_std = np.ones(state_size)
        self.state_count = 0


    def remember(self, state, action, reward, next_state, done):
        # Use the add method of PrioritizedReplayBuffer
        self.memory.add(state, action, reward, next_state, done)

    def normalize_state(self, state):
        return (state - self.state_mean) / (self.state_std + 1e-8) # Add epsilon for stability

    def update_state_stats(self, state):
        # Simple running mean and variance update
        self.state_count += 1
        delta = state - self.state_mean
        self.state_mean += delta / self.state_count
        M2 = np.mean((state - self.state_mean)**2) * (self.state_count -1) if self.state_count > 1 else 0
        self.state_std = np.sqrt(M2 / self.state_count) if self.state_count > 1 else np.ones(self.state_size)


    def act(self, state):
        # Update state statistics before normalization
        self.update_state_stats(np.array(state))
        normalized_state = self.normalize_state(np.array(state))

        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(normalized_state).unsqueeze(0).to(self.device)
        self.qnetwork.eval() # Set network to evaluation mode
        with torch.no_grad():
            q_values = self.qnetwork(state)
        self.qnetwork.train() # Set network back to training mode
        return torch.argmax(q_values).item()

    def replay(self):
        if len(self.memory.experience) < self.batch_size: # Check if enough experiences are in the buffer
            return

        # Use sample method of PrioritizedReplayBuffer which returns weights and indices
        states, actions, rewards, next_states, dones, weights, indices = self.memory.sample(self.batch_size, self.beta)

        # Normalize states and next_states before converting to tensors
        normalized_states = [self.normalize_state(np.array(s)) for s in states]
        normalized_next_states = [self.normalize_state(np.array(s)) for s in next_states]

        states = torch.FloatTensor(normalized_states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(normalized_next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        weights = torch.FloatTensor(weights).to(self.device)


        q_values = self.qnetwork(states).gather(1, actions.unsqueeze(1)).squeeze(1)

        # Double DQN update
        next_actions = self.qnetwork(next_states).max(1)[1].unsqueeze(1)
        next_q_values = self.target_network(next_states).gather(1, next_actions).squeeze(1)

        targets = rewards + (self.gamma * next_q_values * (1 - dones))

        # Compute TD errors and update priorities
        td_errors = targets - q_values.detach() # Use detach() to avoid backprop through target
        self.memory.update_priorities(indices, td_errors.abs().cpu().detach().numpy())

        # Apply importance sampling weights to the loss
        loss = (nn.MSELoss(reduction='none')(q_values, targets) * weights).mean()


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        self.beta = min(1.0, self.beta + self.beta_increment_per_frame) # Anneal beta

    def update_target_network(self):
        self.target_network.load_state_dict(self.qnetwork.state_dict())

# Define state and action sizes
env = InventoryEnv()
state_size = len(env._get_state()) # Get the size of the state vector from the environment
action_size = 11 ** 3  # 3 products, 11 discrete actions (0, 10, ..., 100) each

# Re-initialize agent with new class definition
agent = DQNAgent(state_size, action_size)

episodes = 500
target_update_freq = 10

for e in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action_idx = agent.act(state)

        # Convert flat index to 3 product orders
        orders = np.unravel_index(action_idx, (11, 11, 11))
        orders = [o * 10 for o in orders]

        next_state, reward, done, _ = env.step(orders)
        agent.remember(state, action_idx, reward, next_state, done)
        agent.replay()
        state = next_state
        total_reward += reward

    if e % target_update_freq == 0:
        agent.update_target_network()

    # print(f"Episode {e}, Total Reward: {total_reward}") # Optionally keep print for monitoring

---------------------------------------------------------------

In [17]:
import random
import torch
import numpy as np

class SumTree:
    """
    A SumTree data structure for efficient storage and sampling of priorities.
    Used internally by PrioritizedReplayBuffer.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.data_pointer = 0

    def add(self, priority, data):
        """Add data and update the tree."""
        tree_index = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data
        self.update(tree_index, priority)
        self.data_pointer += 1
        if self.data_pointer >= self.capacity:
            self.data_pointer = 0

    def update(self, tree_index, priority):
        """Update the priority of a data point."""
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        while tree_index != 0:
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change

    def get_leaf(self, v):
        """Retrieve a data point and its priority from the tree."""
        parent_index = 0
        while True:
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            else:
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
        data_index = leaf_index - self.capacity + 1
        return leaf_index, self.tree[leaf_index], self.data[data_index]

    @property
    def total_priority(self):
        """Get the sum of all priorities."""
        return self.tree[0]

class PrioritizedReplayBuffer:
    """
    Prioritized Experience Replay Buffer for DQN.
    """
    def __init__(self, buffer_size, alpha=0.6, seed=42):
        self.sum_tree = SumTree(buffer_size)
        self.alpha = alpha
        self.buffer_size = buffer_size
        self.experience = [None] * buffer_size  # Use a list to store experiences
        self.current_size = 0 # To track the actual number of experiences stored
        self.random_state = np.random.RandomState(seed)
        self.max_priority = 1.0 # Initial max priority for new experiences

    def add(self, state, action, reward, next_state, done):
        """Adds an experience to the buffer with maximum priority."""
        experience = (state, action, reward, next_state, done)
        self.sum_tree.add(self.max_priority, self.current_size) # Store index in SumTree
        self.experience[self.current_size] = experience # Store experience in list
        self.current_size = (self.current_size + 1) % self.buffer_size

    def sample(self, batch_size, beta=0.4):
        """Samples a batch of experiences based on priorities."""
        minibatch = []
        indices = []
        weights = []
        total_priority = self.sum_tree.total_priority
        segment = total_priority / batch_size

        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            v = self.random_state.uniform(a, b)
            tree_index, priority, data_index = self.sum_tree.get_leaf(v) # data_index is the index in the experience list

            # Calculate importance sampling weight
            sampling_probability = priority / total_priority
            weight = (self.buffer_size * sampling_probability) ** -beta
            weights.append(weight)
            indices.append(tree_index) # Store the tree index for priority updates
            minibatch.append(self.experience[data_index]) # Retrieve experience from list

        # Normalize weights
        max_weight = max(weights) if weights else 1.0
        weights = [w / max_weight for w in weights]

        states, actions, rewards, next_states, dones = zip(*minibatch)

        return (torch.FloatTensor(states),
                torch.LongTensor(actions),
                torch.FloatTensor(rewards),
                torch.FloatTensor(next_states),
                torch.FloatTensor(dones),
                torch.FloatTensor(weights),
                indices)

    def update_priorities(self, tree_indices, td_errors):
        """Updates the priorities of sampled experiences based on TD errors."""
        for tree_index, td_error in zip(tree_indices, td_errors):
            priority = abs(td_error) ** self.alpha
            self.sum_tree.update(tree_index, priority)
            self.max_priority = max(self.max_priority, priority) # Update max priority

In [18]:
class DQNAgent:
    def __init__(self, state_size, action_size, gamma=0.99, lr=5e-4,
                 batch_size=128, buffer_size=100000, epsilon_start=1.0,
                 epsilon_end=0.01, epsilon_decay=0.99, alpha=0.6, beta_start=0.4, beta_frames=1000):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.batch_size = batch_size
        self.memory = PrioritizedReplayBuffer(buffer_size, alpha)
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay

        self.beta = beta_start
        self.beta_increment_per_frame = (1.0 - beta_start) / beta_frames

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork = DuelingQNetwork(state_size, action_size).to(self.device)
        self.target_network = DuelingQNetwork(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.qnetwork.parameters(), lr=lr)

    def remember(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        self.qnetwork.eval()
        with torch.no_grad():
            q_values = self.qnetwork(state)
        self.qnetwork.train()
        return torch.argmax(q_values).item()

    def replay(self):
        if self.memory.current_size < self.batch_size: # Check if enough experiences are in the buffer
             return

        states, actions, rewards, next_states, dones, weights, indices = self.memory.sample(self.batch_size, self.beta)

        # Move tensors to the correct device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        weights = weights.to(self.device)

        q_values = self.qnetwork(states).gather(1, actions.unsqueeze(1)).squeeze(1)

        next_actions = self.qnetwork(next_states).max(1)[1].unsqueeze(1)
        next_q_values = self.target_network(next_states).gather(1, next_actions).squeeze(1)

        targets = rewards + (self.gamma * next_q_values * (1 - dones))

        td_errors = targets - q_values.detach()
        # Apply .detach() before converting to numpy
        self.memory.update_priorities(indices, td_errors.abs().cpu().detach().numpy())

        loss = (nn.MSELoss(reduction='none')(q_values, targets) * weights).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        self.beta = min(1.0, self.beta + self.beta_increment_per_frame)

    def update_target_network(self):
        self.target_network.load_state_dict(self.qnetwork.state_dict())

# Re-initialize agent with new class definition
agent = DQNAgent(state_size, action_size)

episodes = 500
target_update_freq = 10

for e in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action_idx = agent.act(state)

        # Convert flat index to 3 product orders
        orders = np.unravel_index(action_idx, (11, 11, 11))
        orders = [o * 10 for o in orders]

        next_state, reward, done, _ = env.step(orders)
        agent.remember(state, action_idx, reward, next_state, done)
        agent.replay()
        state = next_state
        total_reward += reward

    if e % target_update_freq == 0:
        agent.update_target_network()

    # print(f"Episode {e}, Total Reward: {total_reward}")

In [19]:


# Re-run the training loop
episodes = 500
target_update_freq = 10

for e in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action_idx = agent.act(state)

        # Convert flat index to 3 product orders
        orders = np.unravel_index(action_idx, (11, 11, 11))
        orders = [o * 10 for o in orders]

        next_state, reward, done, _ = env.step(orders)
        agent.remember(state, action_idx, reward, next_state, done)
        agent.replay()
        state = next_state
        total_reward += reward

    if e % target_update_freq == 0:
        agent.update_target_network()

    print(f"Episode {e}, Total Reward: {total_reward}")

Episode 0, Total Reward: -2259.0
Episode 1, Total Reward: -2037.1250000000002
Episode 2, Total Reward: -2842.925
Episode 3, Total Reward: -2232.425
Episode 4, Total Reward: -2333.0000000000005
Episode 5, Total Reward: -1991.8499999999995
Episode 6, Total Reward: -2107.6249999999995
Episode 7, Total Reward: -2302.5499999999993
Episode 8, Total Reward: -2421.0750000000007
Episode 9, Total Reward: -2554.5750000000003
Episode 10, Total Reward: -1903.1000000000006
Episode 11, Total Reward: -2390.7499999999995
Episode 12, Total Reward: -2130.0
Episode 13, Total Reward: -2781.3999999999983
Episode 14, Total Reward: -2441.3749999999995
Episode 15, Total Reward: -2463.4
Episode 16, Total Reward: -2374.5499999999997
Episode 17, Total Reward: -2463.4749999999995
Episode 18, Total Reward: -2511.35
Episode 19, Total Reward: -2314.0499999999997
Episode 20, Total Reward: -2096.8750000000005
Episode 21, Total Reward: -2022.4999999999995
Episode 22, Total Reward: -3023.4500000000003
Episode 23, Total R

In [20]:
import random
import torch
import numpy as np

class SumTree:
    """
    A SumTree data structure for efficient storage and sampling of priorities.
    Used internally by PrioritizedReplayBuffer.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.data_pointer = 0

    def add(self, priority, data):
        """Add data and update the tree."""
        tree_index = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data
        self.update(tree_index, priority)
        self.data_pointer += 1
        if self.data_pointer >= self.capacity:
            self.data_pointer = 0

    def update(self, tree_index, priority):
        """Update the priority of a data point."""
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        while tree_index != 0:
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change

    def get_leaf(self, v):
        """Retrieve a data point and its priority from the tree."""
        parent_index = 0
        while True:
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            else:
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
        data_index = leaf_index - self.capacity + 1
        return leaf_index, self.tree[leaf_index], self.data[data_index]

    @property
    def total_priority(self):
        """Get the sum of all priorities."""
        return self.tree[0]

class PrioritizedReplayBuffer:
    """
    Prioritized Experience Replay Buffer for DQN.
    """
    def __init__(self, buffer_size, alpha=0.6, seed=42):
        self.sum_tree = SumTree(buffer_size)
        self.alpha = alpha
        self.buffer_size = buffer_size
        self.experience = [None] * buffer_size  # Use a list to store experiences
        self.current_size = 0 # To track the actual number of experiences stored
        self.random_state = np.random.RandomState(seed)
        self.max_priority = 1.0 # Initial max priority for new experiences

    def add(self, state, action, reward, next_state, done):
        """Adds an experience to the buffer with maximum priority."""
        experience = (state, action, reward, next_state, done)
        self.sum_tree.add(self.max_priority, self.current_size) # Store index in SumTree
        self.experience[self.current_size] = experience # Store experience in list
        self.current_size = (self.current_size + 1) % self.buffer_size

    def sample(self, batch_size, beta=0.4):
        """Samples a batch of experiences based on priorities."""
        minibatch = []
        indices = []
        weights = []
        total_priority = self.sum_tree.total_priority
        segment = total_priority / batch_size

        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            v = self.random_state.uniform(a, b)
            tree_index, priority, data_index = self.sum_tree.get_leaf(v) # data_index is the index in the experience list

            # Calculate importance sampling weight
            sampling_probability = priority / total_priority
            weight = (self.buffer_size * sampling_probability) ** -beta
            weights.append(weight)
            indices.append(tree_index) # Store the tree index for priority updates
            minibatch.append(self.experience[data_index]) # Retrieve experience from list

        # Normalize weights
        max_weight = max(weights) if weights else 1.0
        weights = [w / max_weight for w in weights]

        states, actions, rewards, next_states, dones = zip(*minibatch)

        return (torch.FloatTensor(states),
                torch.LongTensor(actions),
                torch.FloatTensor(rewards),
                torch.FloatTensor(next_states),
                torch.FloatTensor(dones),
                torch.FloatTensor(weights),
                indices)

    def update_priorities(self, tree_indices, td_errors):
        """Updates the priorities of sampled experiences based on TD errors."""
        for tree_index, td_error in zip(tree_indices, td_errors):
            priority = abs(td_error) ** self.alpha
            self.sum_tree.update(tree_index, priority)
            self.max_priority = max(self.max_priority, priority) # Update max priority

In [21]:
# # Calculate and display the average reward per episode
# episode_rewards = []

# for e in range(episodes):
#     state = env.reset()
#     total_reward = 0
#     done = False

#     while not done:
#         action_idx = agent.act(state)
#         orders = np.unravel_index(action_idx, (11, 11, 11))
#         orders = [o * 10 for o in orders]
#         next_state, reward, done, _ = env.step(orders)
#         state = next_state
#         total_reward += reward

#     episode_rewards.append(total_reward)

# average_reward = np.mean(episode_rewards)
# print(f"Average Reward over {episodes} episodes: {average_reward:.2f}")

In [22]:
# 'agent' is the trained DQNAgent instance
model_path = 'dueling_dqn_policy_net.pth'
torch.save(agent.qnetwork.state_dict(), model_path)

print(f"Policy network saved to {model_path}")

Policy network saved to dueling_dqn_policy_net.pth


----------------------------------------------------


In [23]:
# rl agent

#rl_agent.py
#import gym
import subprocess
import sys
try:
    import torch
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torch"])
import torch
import torch.nn as nn
import numpy as np
import os





# Path to the model file - Changed to the saved policy network filename
model_path =  "./dueling_dqn_policy_net.pth"


# Updated to DuelingQNetwork to match the trained model
class DuelingQNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(DuelingQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

        self.value_stream = nn.Linear(hidden_size, 1)
        self.advantage_stream = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        value = self.value_stream(x)
        advantage = self.advantage_stream(x)

        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values


class RLAgent:
    def __init__(self):
        pass

    def flatten_state(self, state):

      # idea is to convert the environment state into a flat numpy array that the neural network can process

      if isinstance(state, dict):
          return np.concatenate([np.array(v, dtype=np.float32) for v in state.values()])
      return np.array(state, dtype=np.float32)

    def run_policy(self,state):
        ''' policy execution function '''
        state = self.flatten_state(state)
        STATE_SIZE = len(state)
        ACTION_SIZE = 11 ** 3  # 3 products, 11 discrete actions each

        # Load the DuelingQNetwork model
        policy_net = DuelingQNetwork(STATE_SIZE, ACTION_SIZE)
        policy_net.load_state_dict(torch.load(model_path,map_location='cpu'))

        # loading a pretrained DQN model


        policy_net.eval()
        state_tensor = torch.FloatTensor(state).unsqueeze(0)

        with torch.no_grad():
            q_values = policy_net(state_tensor)
        action_idx = torch.argmax(q_values).item()

        # Convert flat index → orders for 3 products
        orders = np.unravel_index(action_idx, (11, 11, 11))
        return [o * 10 for o in orders]  # since action space is {0,10,...,100}